# Evaluation Criteria

This Colab notebook demonstrates how you can evaluate/test your Agents using various criteria using CLI. It covers following criteria:

  *   `tool_trajectory_avg_score`: Compares agent's tool calls with expected trajectory.
  *   `response_match_score`: Evaluates final response similarity using Rouge-1.
  *   `final_response_match_v2`: Assesses semantic equivalence of responses using an LLM judge.
  *   `rubric_based_final_response_quality_v1`: Evaluates response quality against user-defined rubrics with an LLM judge.
  *   `rubric_based_tool_use_quality_v1`: Assesses tool usage quality against user-defined rubrics with an LLM judge.
  *   `hallucinations_v1`: Checks for false, contradictory, or unsupported claims in agent responses.
  *   `safety_v1`: Evaluates the harmlessness of agent responses using Vertex AI General AI Eval SDK.

## Evaluation Criteria

### Criterion - `tool_trajectory_avg_score`

This criterion compares the tool call trajectory produced by the agent with an
expected trajectory and computes an average score based on exact match.

#### Details

For each invocation that is being evaluated, this criterion compares the list of
tool calls produced by the agent against the list of expected tool calls. The
comparison is done by performing an exact match on the tool name and tool
arguments for each tool call in the list. If all tool calls in an invocation
match exactly in content and order, a score of 1.0 is awarded for that
invocation, otherwise the score is 0.0. The final value is the average of these
scores across all invocations in the eval case.

#### Output And How To Interpret

The output is a score between 0.0 and 1.0, where 1.0 indicates a perfect match
between actual and expected tool trajectories for all invocations, and 0.0
indicates a complete mismatch for all invocations. Higher scores are better. A
score below 1.0 means that for at least one invocation, the agent's tool call
trajectory deviated from the expected one.

More details can be found [here](https://google.github.io/adk-docs/evaluate/criteria/#tool_trajectory_avg_score).

Takes about ~30 seconds to run the cell below.

In [12]:
eval_config = """
{
  "criteria": {
    "tool_trajectory_avg_score": 1.0
  }
}
"""

with open("./notion_mcp_agent/eval_config.json", "w") as f:
    f.write(eval_config)


In [ ]:
!adk eval ./notion_mcp_agent --config_file_path ./notion_mcp_agent/eval_config.json ./notion_mcp_agent/evalset604380.evalset.json --print_detailed_results --log_level=CRITICAL

Using evaluation criteria: criteria={'tool_trajectory_avg_score': 1.0, 'response_match_score': 0.8} user_simulator_config=None
*********************************************************************
Eval Run Summary
../notion_mcp_agent/evalset1d0118:
  Tests passed: 0
  Tests failed: 1
********************************************************************
Eval Set Id: ../notion_mcp_agent/evalset1d0118
Eval Id: caseedb6bf
Overall Eval Status: FAILED
---------------------------------------------------------------------
Metric: tool_trajectory_avg_score, Status: FAILED, Score: 0.6, Threshold: 1.0
---------------------------------------------------------------------
Metric: response_match_score, Status: FAILED, Score: 0.2030398835150601, Threshold: 0.8
---------------------------------------------------------------------
Invocation Details:
+----+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------

C:\Users\91789\Desktop\Project\ADK_Project\.venv\Lib\site-packages\google\adk\evaluation\metric_evaluator_registry.py:90: UserWarning: [EXPERIMENTAL] MetricEvaluatorRegistry: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  metric_evaluator_registry = MetricEvaluatorRegistry()
C:\Users\91789\Desktop\Project\ADK_Project\.venv\Lib\site-packages\google\adk\evaluation\local_eval_service.py:79: UserWarning: [EXPERIMENTAL] UserSimulatorProvider: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  user_simulator_provider: UserSimulatorProvider = UserSimulatorProvider(),
C:\Users\91789\Desktop\Project\ADK_Project\.venv\Lib\site-packages\google\adk\cli\cli_tools_click.py:650: UserWarning: [EXPERIMENTAL] UserSimulatorProvider: This feature is experimental and may change or be removed in future versions without notic

### Criterion - `response_match_score`

This criterion evaluates if agent's final response matches a golden/expected
final response using Rouge-1.

#### Details

To learn more, see details on
[ROUGE-1](https://github.com/google-research/google-research/tree/master/rouge).

#### Output And How To Interpret

Value range for this criterion is [0,1], with values closer to 1 more desirable.

More details can be found [here](https://google.github.io/adk-docs/evaluate/criteria/#response_match_score).

Takes about ~30 seconds to run the cell below.

In [ ]:
eval_config="""
{
  "criteria": {
    "response_match_score": 0.8
  }
}
"""

with open("./notion_mcp_agent/eval_config.json", "w") as f:
    f.write(eval_config)

In [ ]:
!adk eval ./notion_mcp_agent --config_file_path ./notion_mcp_agent/eval_config ./notion_mcp_agent/evalset1d0118 --print_detailed_results --log_level=CRITICAL

### Criterion - `final_response_match_v2`

This criterion evaluates if the agent's final response matches a golden/expected
final response using LLM as a judge.

#### Details

This criterion uses a Large Language Model (LLM) as a judge to determine if the
agent's final response is semantically equivalent to the provided reference
response. It is designed to be more flexible than lexical matching metrics (like
`response_match_score`), as it focuses on whether the agent's response contains
the correct information, while tolerating differences in formatting, phrasing,
or the inclusion of additional correct details.

For each invocation, the criterion prompts a judge LLM to rate the agent's
response as "valid" or "invalid" compared to the reference. This is repeated
multiple times for robustness (configurable via `num_samples`), and a majority
vote determines if the invocation receives a score of 1.0 (valid) or 0.0
(invalid). The final criterion score is the fraction of invocations deemed valid
across the entire eval case.

#### Output And How To Interpret

The criterion returns a score between 0.0 and 1.0. A score of 1.0 means the LLM
judge considered the agent's final response to be valid for all invocations,
while a score closer to 0.0 indicates that many responses were judged as invalid
when compared to the reference responses. Higher values are better.

More details can be found [here](https://google.github.io/adk-docs/evaluate/criteria/#final_response_match_v2).

Takes about ~1 minute to run the cell below.

In [ ]:
eval_config="""
{
  "criteria": {
    "final_response_match_v2": {
      "threshold": 0.8,
      "judge_model_options": {
            "judge_model": "gemini-2.5-flash",
            "num_samples": 5
      }
    }
  }
}
"""

with open("./notion_mcp_agent/eval_config.json", "w") as f:
    f.write(eval_config)

In [ ]:
!adk eval ../notion_mcp_agent ../notion_mcp_agent/evalset604380 --config_file_path ../notion_mcp_agent/eval_config --print_detailed_results --log_level=CRITICAL

### Criterion - `rubric_based_final_response_quality_v1`

This criterion assesses the quality of an agent's final response against a
user-defined set of rubrics using LLM as a judge.

#### Details

This criterion provides a flexible way to evaluate response quality based on
specific criteria that you define as rubrics. For example, you could define
rubrics to check if a response is concise, if it correctly infers user intent,
or if it avoids jargon.

The criterion uses an LLM-as-a-judge to evaluate the agent's final response
against each rubric, producing a `yes` (1.0) or `no` (0.0) verdict for each.
Like other LLM-based metrics, it samples the judge model multiple times per
invocation and uses a majority vote to determine the score for each rubric in
that invocation. The overall score for an invocation is the average of its
rubric scores. The final criterion score for the eval case is the average of
these overall scores across all invocations.

#### Output And How To Interpret

The criterion outputs an overall score between 0.0 and 1.0, where 1.0 indicates
that the agent's responses satisfied all rubrics across all invocations, and 0.0
indicates that no rubrics were satisfied. The results also include detailed
per-rubric scores for each invocation. Higher values are better.

More details can be found [here](https://google.github.io/adk-docs/evaluate/criteria/#rubric_based_final_response_quality_v1).

Takes about 1-2 minutes to run the cell below.

In [ ]:
eval_config="""
{
  "criteria": {
    "rubric_based_final_response_quality_v1": {
      "threshold": 0.8,
      "judge_model_options": {
        "judge_model": "gemini-2.5-flash",
        "num_samples": 5
      },
      "rubrics": [
        {
          "rubric_id": "conciseness",
          "rubric_content": {
            "text_property": "The response from the agent is direct and to the point."
          }
        },
        {
          "rubric_id": "intent_inference",
          "rubric_content": {
            "text_property": "The response from the agent accurately infers the underlying goal from ambiguous queries."
          }
        }
      ]
    }
  }
}
"""

with open("./notion_mcp_agent/eval_config.json", "w") as f:
    f.write(eval_config)

In [ ]:
!adk eval ./notion_mcp_agent --config_file_path ./notion_mcp_agent/eval_config.json ./notion_mcp_agent/evalset604380.evalset.json --print_detailed_results --log_level=CRITICAL

### Criterion - `hallucinations_v1`
This criterion assesses whether a model response contains any false,
contradictory, or unsupported claims.

#### Details

This criterion assesses whether a model response contains any false,
contradictory, or unsupported claims based on context that includes developer
instructions, user prompt, tool definitions, and tool invocations and their
results. It uses LLM-as-a-judge and follows a two-step process:

1.  **Segmenter**: Segments the agent response into individual sentences.
2.  **Sentence Validator**: Evaluates each segmented sentence against the
    provided context for grounding. Each sentence is labeled as `supported`,
    `unsupported`, `contradictory`, `disputed` or `not_applicable`.

The metric computes an Accuracy Score: the percentage of sentences that are
`supported` or `not_applicable`. By default, only the final response is
evaluated. If `evaluate_intermediate_nl_responses` is set to true in the
criterion, intermediate natural language responses from agents are also
evaluated.

#### Output And How To Interpret

The criterion returns a score between 0.0 and 1.0. A score of 1.0 means all
sentences in agent's response are grounded in the context, while a score closer
to 0.0 indicates that many sentences are false, contradictory, or unsupported.
Higher values are better.

More details can be found [here](https://google.github.io/adk-docs/evaluate/criteria/#hallucinations_v1).

Takes about 1-2 minutes to run the cell below.

In [ ]:
eval_config="""
{
  "criteria": {
    "hallucinations_v1": {
      "threshold": 0.5,
      "evaluate_intermediate_nl_responses": false
    }
  }
}
"""

with open("./notion_mcp_agent/eval_config.json", "w") as f:
    f.write(eval_config)

In [ ]:
!adk eval ./notion_mcp_agent --config_file_path ./notion_mcp_agent/eval_config.json ./notion_mcp_agent/evalset604380.evalset.json --print_detailed_results --log_level=CRITICAL

### Criterion - `safety_v1`

This criterion evaluates the safety (harmlessness) of an Agent's Response.

#### Details

This criterion assesses whether the agent's response contains any harmful
content, such as hate speech, harassment, or dangerous information. Unlike other
metrics implemented natively within ADK, `safety_v1` delegates the evaluation to
the Vertex AI General AI Eval SDK.

#### Output And How To Interpret

The criterion returns a score between 0.0 and 1.0. Scores closer to 1.0 indicate
that the response is safe, while scores closer to 0.0 indicate potential safety
issues.

More details can be found [here](https://google.github.io/adk-docs/evaluate/criteria/#safety_v1).

Takes about 1-2 minutes to run the cell below.

In [ ]:
eval_config="""
{
  "criteria": {
    "safety_v1": 0.8
  }
}
"""

with open("./notion_mcp_agent/eval_config.json", "w") as f:
    f.write(eval_config)

In [ ]:
!adk eval ./notion_mcp_agent --config_file_path ./notion_mcp_agent/eval_config.json ./notion_mcp_agent/evalset604380.evalset.json --print_detailed_results --log_level=CRITICAL